## Financial Research Assistant with Persistent Memory

Multi-agent MCPs that researches companies and stores knowledge in a db

In [ ]:
import os
import json
from datetime import datetime
from dotenv import load_dotenv
from agents.mcp import MCPServerStdio
from contextlib import AsyncExitStack
from agents import Agent, Runner, trace, Tool
from IPython.display import display, Markdown

load_dotenv(override=True)

In [ ]:
research_server_path = os.path.abspath("research_server.py")
research_params = {"command": "python", "args": [research_server_path]}

brave_env = {"BRAVE_API_KEY": os.getenv("BRAVE_API_KEY")}
brave_params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-brave-search"], "env": brave_env}

sandbox_path = os.path.abspath("sandbox")
files_params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-filesystem", sandbox_path]}

In [ ]:
async def get_researcher(mcp_servers) -> Agent:
    instructions = f"""You research companies using web search tools.
Your job is to gather facts about the company: business model, products, recent news, financial metrics.
After researching, you must save your findings using the save_company and save_research tools.
Always save the company information first with symbol, name, and sector.
Then save detailed research findings.
Current date: {datetime.now().strftime("%Y-%m-%d")}
"""
    return Agent(
        name="Researcher",
        instructions=instructions,
        model="gpt-4.1-mini",
        mcp_servers=mcp_servers
    )

async def get_researcher_tool(mcp_servers) -> Tool:
    researcher = await get_researcher(mcp_servers)
    return researcher.as_tool(
        tool_name="Researcher",
        tool_description="Research companies and save findings. Provide company symbol and what to research."
    )

In [ ]:
async def get_analyst(mcp_servers) -> Agent:
    instructions = """You analyze companies and generate investment theses.
First, read existing research using the research://company/{symbol} resource.
Synthesize the research into a clear investment thesis covering strengths, risks, and outlook.
Save your thesis using the save_thesis tool.
Create a markdown report and save it to a file.
"""
    return Agent(
        name="Analyst",
        instructions=instructions,
        model="gpt-4.1-mini",
        mcp_servers=mcp_servers
    )

async def get_analyst_tool(mcp_servers) -> Tool:
    analyst = await get_analyst(mcp_servers)
    return analyst.as_tool(
        tool_name="Analyst",
        tool_description="Analyze company research and generate investment thesis. Provide company symbol."
    )

In [ ]:
async def run_research_assistant(query: str):
    async with AsyncExitStack() as stack:
        research_server = await stack.enter_async_context(
            MCPServerStdio(research_params, client_session_timeout_seconds=60)
        )
        
        brave_server = await stack.enter_async_context(
            MCPServerStdio(brave_params, client_session_timeout_seconds=60)
        )
        
        files_server = await stack.enter_async_context(
            MCPServerStdio(files_params, client_session_timeout_seconds=60)
        )
        
        researcher_tool = await get_researcher_tool([research_server, brave_server])
        analyst_tool = await get_analyst_tool([research_server, files_server])
        
        main_instructions = """You coordinate research and analysis of companies.
When asked to research a company:
1. Use the Researcher tool to gather and save information
2. Use the Analyst tool to create an investment thesis
3. Report back with a summary

If the company already has research in the database, mention that and still run fresh analysis.
"""
        
        main_agent = Agent(
            name="ResearchCoordinator",
            instructions=main_instructions,
            model="gpt-4.1-mini",
            tools=[researcher_tool, analyst_tool]
        )
        
        with trace("research_assistant"):
            result = await Runner.run(main_agent, query, max_turns=30)
            display(Markdown(result.final_output))

### Research a company for the first time

In [ ]:
await run_research_assistant("Research NVIDIA (NVDA) and provide an investment analysis")

### Check what we have stored

In [ ]:
async with MCPServerStdio(research_params, client_session_timeout_seconds=60) as server:
    result = await server.session.read_resource("research://company/NVDA")
    for content in result.contents:
        print(content.text)

### Research another company

In [ ]:
await run_research_assistant("Research Apple (AAPL) and analyze their business")

### View all stored companies

In [ ]:
async with MCPServerStdio(research_params, client_session_timeout_seconds=60) as server:
    result = await server.session.read_resource("research://all")
    for content in result.contents:
        data = json.loads(content.text)
        print(f"Total companies: {data['total_companies']}")
        
        for company in data['companies']:
            print(f"- {company['symbol']}: {company['name']} ({company['sector']})")

### Re-analyze NVIDIA showing memory persistence

In [ ]:
await run_research_assistant("Update the analysis for NVIDIA with latest information")